In [1]:
import astropy.io.fits as fits
import numpy as np
import os, sys

In [80]:
fits_file = "C:/Users/Willi/Downloads/RemoteAstrophotography-com-NGC2070-narrowband/Tarantula Nebula-oiii.fit"
fits.open(fits_file)

In [81]:
with fits.open(fits_file) as hdul:
    hdul.info()
    data = hdul[0].data

Filename: C:/Users/Willi/Downloads/RemoteAstrophotography-com-NGC2070-narrowband/Tarantula Nebula-oiii.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     113   (3056, 3056)   float32   


In [27]:
print(data.shape) # dimensions of the image
height, width = data.shape

(3056, 3056)


In [28]:
x_center = width // 2
y_center = height //2
print(x_center, y_center)

1528 1528


In [29]:
pixel_value = data[1529, 1529]
print(pixel_value)

4216.541


In [8]:
# Get the median:
image = fits.getdata(fits_file)
median = np.median(image)
print(median)

117.57136


In [68]:
# Define the radius from the center of the image to the edge
radius = np.arange(0, width+1, 1)
print(radius)

[   0    1    2 ... 3054 3055 3056]


In [ ]:
ys, xs, = np.indices(image.shape)
r1 = np.sqrt((ys-y_center)**2+(xs-x_center)**2)

In [73]:
test = np.zeros((10, 2))
test[0] = [1, 2]
print(test)

[[1. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [83]:
theta = np.linspace(0, 2*np.pi, 100)
indices = np.zeros((theta.size, 2))
overall = np.zeros((len(radius), 2)) # 2D array with the radius and average pixel value
pixel_values = []
for j in range(len(radius)):
    r = radius[j]

    for i in range(theta.size):
        x = x_center + r*np.cos(theta[i])
        y = y_center + r*np.sin(theta[i])
        point = [y, x]
        # indices.append(point)
        indices[i] = [y, x]
        pixel_value = data[int(y), int(x)]
        pixel_values.append(pixel_value)
        mean_pixel_value = np.mean(pixel_values)

    overall[j] = [r, mean_pixel_value]

print(overall)

IndexError: index 3056 is out of bounds for axis 1 with size 3056

In [78]:
print(np.sqrt((indices[:,0]-x_center)**2+(indices[:,1]-y_center)**2))
print(min(indices[:,0]))

[10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
1518.0012587232613


In [ ]:
# Find the intensity at a given radius from the center of the image
# Find a way to calculate radius all the way out to the corners, and exclude points that would be outside the edge

In [ ]:
hdul.close() # cldse the file

Plotting script from Dimitri

In [ ]:
def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile

In [87]:
np.bincount?

Docstring:
bincount(x, weights=None, minlength=0)

Count number of occurrences of each value in array of non-negative ints.

The number of bins (of size 1) is one larger than the largest value in
`x`. If `minlength` is specified, there will be at least this number
of bins in the output array (though it will be longer if necessary,
depending on the contents of `x`).
Each bin gives the number of occurrences of its index value in `x`.
If `weights` is specified the input array is weighted by it, i.e. if a
value ``n`` is found at position ``i``, ``out[n] += weight[i]`` instead
of ``out[n] += 1``.

Parameters
----------
x : array_like, 1 dimension, nonnegative ints
    Input array.
weights : array_like, optional
    Weights, array of the same shape as `x`.
minlength : int, optional
    A minimum number of bins for the output array.

    .. versionadded:: 1.6.0

Returns
-------
out : ndarray of ints
    The result of binning the input array.
    The length of `out` is equal to ``np.amax(x)+1`

In [1]:
import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic

In [2]:
def radial_profile(data, center):
    """ Calculate the radial profile of a 2D array. """
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)

    # Define the desired thickness
    thickness = 4  # thickness in pixels
    
    # Bin the pixel values by their distances
    r_flat = r.ravel()
    image_flat = data.ravel()
    # Define bin edges with the desired thickness
    bin_edges = np.arange(0, r.max() + thickness, thickness)

    # Calculate the mean intensity within each bin
    radial_profile, bin_edges, binnumber = binned_statistic(r_flat, image_flat, statistic='mean', bins=bin_edges)

    # Calculate bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    return radial_profile, bin_centers


In [3]:
def process_fits_file(file_path):
    """ Process a single FITS file and return the radial profiles. """
    with fits.open(file_path) as hdul:  
        profiles = {}
        bin_centers = {}
        dp = {}
        for i in range(0, len(hdul)):
            dp [f'extension_{i}'] = hdul[i].header['DP']
            data = hdul[i].data
            center = (data.shape[1] // 2, data.shape[0] // 2)
            profiles[f'extension_{i}'], bin_centers[f'extension_{i}'] = radial_profile(data, center)
    return profiles, bin_centers, dp

In [4]:
def process_directory(directory):
    """ Process all FITS files in the given directory and subdirectories. """
    results = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.fits'):
                file_path = os.path.join(root, file)
                profiles, bin_centers, dp = process_fits_file(file_path)
                rel_dir = os.path.relpath(root, directory)
                if rel_dir not in results:
                    results[rel_dir] = []
                results[rel_dir].append((file, profiles, bin_centers, dp))
    return results

In [5]:
def plot_radial_profiles(results):
    """ Plot the radial profiles from the results dictionary. """
    for rel_dir, files_data in results.items():
        for file, profiles, bin_centers, dp in files_data:
            plt.figure(figsize=(10, 6))
            for ext, profile in profiles.items():
                radii = bin_centers[ext] * dp[ext]  # Convert pixel radius to arcseconds
                plt.loglog(radii, profile, label=ext)
            plt.xlabel('Radius (arcseconds)')
            plt.ylabel('Intensity')
            plt.title(f'Radial Profile for {file}')
            plt.legend()
            plt.grid(True)
            plt.show()

# Example usage
current_directory = os.getcwd()
base_directory = current_directory
results = process_directory(base_directory)
plot_radial_profiles(results)

In [6]:
def plot_radial_profiles(results):
    """ Plot the radial profiles from the results dictionary. """
    for rel_dir, files_data in results.items():
        for file, profiles, dp in files_data:
            plt.figure(figsize=(10, 6))
            for ext, profile in profiles.items():
                radii = np.arange(len(profile)) * dp[ext]  # Convert pixel radius to arcseconds
                plt.loglog(radii, profile, label=ext)
            plt.xlabel('Radius (arcseconds)')
            plt.ylabel('Intensity')
            plt.title(f'Radial Profile for {file}')
            plt.legend()
            plt.grid(True)
            plt.show()
plot_radial_profiles(results)